In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from itertools import izip

In [2]:
class FanDuelScraper(object):
    def __init__(self):
        """Create a Firefox Browser to Scrape FanDuel Contest Data."""
        self.ff = webdriver.Firefox(firefox_profile=webdriver.FirefoxProfile('/Users/andingo/Library/Application Support/Firefox/Profiles/eual1qsf.PythonScripts'))
    
    def gotoURL(self, url):
        """Goto the contest url."""
        self.ff.get(url)
    
    def getPrizes(self):
        """Get a list of places and corresponding prizes. For prizes earned by a range of ranks,
        use the place of the lowest ranking position to win that prize."""
        elem = self.ff.find_element_by_class_name("toggle-show-prize-scoring")
        elem.click()
        places = []
        prizes = []
        for elem in self.ff.find_elements_by_tag_name('dl'):
            try:
                spl = elem.text.split('$')
                prize = float(spl[1].replace(',',''))
                place = int(spl[0].split(' - ')[-1][:-3])
#                 print '%6d\t%.2f' % (place, prize)
                places.append(place)
                prizes.append(prize)
            except Exception, e:
                print e.message
        elem = self.ff.find_element_by_class_name('live-prize-scoring-close')
        elem = elem.find_element_by_class_name('button')
        elem.click()
        return places, prizes
    
    def getLevels(self, places, prizes):
        page_num = 0
        d = dict(Prize=[], Score=[])
        for k, rk in enumerate(places):
            new_page_num = rk/10+int(rk % 10 != 0)
            if new_page_num != page_num:
                page_num = new_page_num
                elem = self.ff.find_element_by_class_name('paging-input')
                elem.send_keys('%s%d' % (10*Keys.BACKSPACE, page_num))
                time.sleep(2)
                winnings = map(lambda x: float(x.text[1:].replace(',','')), self.ff.find_elements_by_class_name('user-winnings'))
                scores = map(lambda x: float(x.text.replace(',','')), self.ff.find_elements_by_class_name('user-score'))
            ind = rk - (page_num - int(rk % 10 != 0))*10 + 10*int(rk % 10 == 0) - 1
            if prizes[k] != winnings[ind]:
                d['Prize'].append(prizes[k])
                d['Score'].append(scores[ind]+.01)
            d['Prize'].append(winnings[ind])
            d['Score'].append(scores[ind])
        #     print rk, prizes[k], winnings[ind], scores[ind]
        assert not any([d['Prize'][i] <= d['Prize'][i+1] for i in range(len(d['Prize'])-1)])
        return pd.DataFrame(d)
    
    def getGames(self):
        elems = self.ff.find_elements_by_class_name('live-fixture__team-code')
        return ['%s@%s' % (a.text,h.text) for a, h in izip(elems[::2], elems[1::2])]
    
    def exit(self):
        self.ff.quit()
        self.ff.close()

In [3]:
fds = FanDuelScraper()

WebDriverException: Message: Can't load the profile. Profile Dir: /var/folders/tl/zzwysg6s4l55j3bwqyddw1b00000gn/T/tmpMTsIAp/webdriver-py-profilecopy If you specified a log_file in the FirefoxBinary constructor, check it for details.


In [ ]:
url = 'https://www.fanduel.com/games/14647/contests/14647-22550651/scoring'
fds.gotoURL(url)
fds.getGames()
places, prizes = fds.getPrizes()
scores = fds.getLevels(places, prizes)
print 'P = ',scores.Prize.tolist()
print 'L = ',scores.Score.tolist()